# 参数管理

考虑在有module的情况下访问或者管理参数。

先考虑一个单隐藏层MLP

In [24]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    nn.Linear(8,1)
)
X = torch.rand(size=(2,4))
print(net(X))

tensor([[0.1241],
        [0.1389]], grad_fn=<AddmmBackward0>)


## 获取一层中的n权重

In [2]:
print(net[2].state_dict())  # 访问第二层的参数

OrderedDict([('weight', tensor([[ 0.1968,  0.2480,  0.3306,  0.1889, -0.2792,  0.0940,  0.2401,  0.1442]])), ('bias', tensor([0.0610]))])


In [3]:
# 访问第二层的偏置参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0610], requires_grad=True)
tensor([0.0610])


In [6]:
print(net[2].weight.grad == None)  # 权重的梯度在反向传播前为空

True


In [7]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [8]:
net.state_dict()['2.bias'].data

tensor([0.0610])

In [9]:
def block1():
    return nn.Sequential(
        nn.Linear(4,8),
        nn.ReLU(),
        nn.Linear(8,4),
        nn.ReLU()
    )
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(
    block2(),
    nn.Linear(4,1)
)

print(rgnet(X))

tensor([[0.1068],
        [0.1068]], grad_fn=<AddmmBackward0>)


In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 修改默认的初始参数

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
print(net[0].weight.data[0])  # 查看第一层的权重参数
print(net[0].bias.data)      # 查看第一层的偏置参数

tensor([-0.0051, -0.0083,  0.0073, -0.0136])
tensor([0., 0., 0., 0., 0., 0., 0., 0.])


In [12]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
        
net.apply(init_constant)
print(net[0].weight.data[0])  # 查看第一层的权重参数
print(net[0].bias.data)      # 查看第一层的偏置参数

tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0.])


In [13]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])  # 查看第一层的权重参数
print(net[2].weight.data)      # 查看第二层的权重参数

tensor([ 0.1444, -0.5032,  0.3079, -0.4662])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
print(net[0].weight.data)  # 查看第一层的权重参数

Init ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
Init ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))
tensor([[ 0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.0000],
        [ 9.0355,  0.0000, -0.0000, -8.7955],
        [ 9.7313, -8.7391,  5.5579,  0.0000],
        [-5.8401,  0.0000, -9.5707, -0.0000],
        [-9.2654,  6.4986, -8.7337,  5.2166],
        [-5.4260, -8.8009,  0.0000,  0.0000]])


In [16]:
# 简单 粗暴的方法
net[0].weight.data[:] += 1.0
net[0].weight.data[0, 0] = 42
print(net[0].weight.data[0])

tensor([42.,  2.,  2.,  2.])


## 参数绑定

e多个层共享一些参数

In [19]:
shared = nn.Linear(8,8)
net = nn.Sequential(
    nn.Linear(4,8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8,1)
)

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])  # True
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])  # True


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

In [ ]:
# 自定义一个无参数的层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
print(layer(torch.FloatTensor([1,2,3,4,5])))  # tensor([-2., -1., 0., 1., 2.])

tensor([-2., -1.,  0.,  1.,  2.])


In [ ]:
net = nn.Sequential(
    nn.Linear(8,128),
    CenteredLayer()
)
Y = net(torch.rand(4,8))
print(Y.mean())  # tensor(-7.4506e-09, grad_fn=<MeanBackward0>)

tensor(-7.4506e-09, grad_fn=<MeanBackward0>)


In [27]:
# 自定义一个有参数的层
class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(out_features,))
        # self.weight = nn.Parameter(torch.zeros(in_features, out_features))
        # self.bias = nn.Parameter(torch.zeros(out_features,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5,3)
print(dense.weight)

Parameter containing:
tensor([[-0.6810, -0.5173,  0.1590],
        [ 1.8465,  1.6802,  0.7742],
        [-0.1727,  1.8331, -1.3380],
        [-0.6608,  0.1719, -0.6723],
        [ 0.9717,  1.3464, -1.4139]], requires_grad=True)


In [28]:
# 使用自定义层i直接执行正向传播计算
print(dense(torch.rand(2,5)))  # tensor([[0., 0.,

tensor([[0.7151, 1.4433, 0.0000],
        [1.3851, 2.3780, 0.0000]])


In [30]:
# 使用自定义的层构建模型
net = nn.Sequential(
    MyLinear(64,8),
    MyLinear(8,1)
)

print(net(torch.rand(2,64)))

tensor([[9.8956],
        [3.6193]])


## 读写文件

In [32]:
x = torch.arange(4)
torch.save(x, 'x.pt')

x2 = torch.load('x.pt', weights_only=True)
print(x2)  # tensor([0, 1, 2, 3])

tensor([0, 1, 2, 3])


In [33]:
y = torch.zeros(4)
torch.save([x,y], 'x.pt')
x2, y2 = torch.load('x.pt', weights_only=True)
print(x2)  # tensor([0, 1, 2, 3])
print(y2)  # tensor([0., 0., 0., 0.

tensor([0, 1, 2, 3])
tensor([0., 0., 0., 0.])


In [35]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'xy.pt')
mydict2 = torch.load('xy.pt', weights_only=True)
print(mydict2['x'])  # tensor([0, 1, 2, 3])
print(mydict2['y'])  # tensor([0., 0., 0., 0.])

tensor([0, 1, 2, 3])
tensor([0., 0., 0., 0.])


In [36]:
# 加载和保存模型参数

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
    
net = MLP()
X = torch.rand(size=(2,20))
Y = net(X)

In [37]:
torch.save(net.state_dict(), 'mlp.params')

In [39]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params', weights_only=True))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
Y_clone = clone(X)
print(Y_clone == Y)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])


: 